# Shopify Technical Challenge- Data Science
## Fall 2022 Data Science Intern Challenge
## Caitlin Malloch
### May 9, 2022

## Question 1: Sneakers Dataset- Shopify

Problem:
On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

Dataset:
- 100 sneaker shops
- 1 shoe model per sneaker shop
- AOV= $3,145.13

Challenge:
1) Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.
2) What metric would you report for this dataset?
3) What is its value?


### 1) Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.

In [ ]:
import matplotlib.pyplot
import pandas as pd

In [24]:
data = pd.read_csv('sneaker_data_shopify.csv')
data.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [7]:
data.shape

(5000, 7)

In [8]:
print(data['created_at'].min())
print(data['created_at'].max())

2017-03-01 0:08:09
2017-03-30 9:55:00


Check that there is no missing values in the data.

In [9]:
data.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

Confirmed that there is no missing data values and that the data is ready to be analyzed.

In [10]:
#summary stats for order_amount column
data.order_amount.describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

Looking at the summary statistics for the order_amount column, we can see that there are 5000 orders and there is a **mean of 3145.13**, which matches the originally reported Average Order Value(AOV). However, we know that something is wrong with that value, and looking at this summary statistics chart, we see there is a **standard deviation of $41,282.54**. That is a very large deviation from the mean, so we need to reflect this in our new calculation. The median order value is the 50% value of $284.00.
It is also important to note that there is a very large value difference between the min. and max. order amount values.
We can likely conclude that the dataset contains outliers that is skewing the AOV calculation.

In [29]:
import plotly.express as px

In [38]:
df = data
fig = px.scatter(df, x="created_at", y="order_amount", color="shop_id", log_x=False, size_max=50,
    hover_data=["payment_method", "total_items"],template='plotly_dark',
    labels=  {
    "created_at": "Time",
    "order_amount": "Order Amount ($)"},
    title="March 2017 Sales Distribution")
fig.show()

With a quick visual scan of this chart, the outliers can be clearly seen. We can see that the shop_id=42 and shop_id=78 are skewing the distribution of sales as the outliers in the dataset. We can also see that shop_id=42 has created the orders with the higher order amount values of ~$704,000 per order. This will not give us an accurate picture of the average order because it is only one store with a much higher volume of sales with a much larger order amount.

The two shops will need to be further analyzed before removing data outliers.

In [41]:
# shop id 42
df_shop_42 = data[data['shop_id'] == 42]
fig = px.scatter(df_shop_42, x="created_at",
                 y="order_amount",
                 color="shop_id",log_x=False,
                 size_max=50, template="plotly_dark",
                 hover_data=["payment_method", "total_items"],
                 labels={
                     "created_at": "Time",
                     "order_amount": "Order Amount (Dollars)",
                 },
                 title="Shop_ID: 42 Sales Distributions" )
fig.show()

In [42]:
# shop id 78
df_shop_78 = data[data['shop_id'] == 78]
fig = px.scatter(df_shop_78, x="created_at",
                 y="order_amount",
                 color="shop_id",log_x=False,
                 size_max=50, template="plotly_dark",
                 hover_data=["payment_method", "total_items"],
                 labels={
                     "created_at": "Time",
                     "order_amount": "Order Amount (Dollars)",
                 },
                 title="Shop_ID: 78 Sales Distributions" )
fig.show()

From the analysis of these two outlier store ID's, we can conclude the following:
Shop ID: 42
    - bulk orders with 2000 items
    - some orders contain 1-3 items
Shop ID: 78
    - typical sales are well over $20,000 so we can see that this particular store is selling expensive sneakers

The bulk orders should be further analyzed because it represents the greatest outlier in the data. We need to know more about the revenue generated by this outlier before moving forward.

### 2) What metric would you report for this dataset?

In this case, AOV is not a great metric to report because the distribution is far too skewed by bulk purchases. AOV also varies wildly based on the store ID, which means it is also not a great metric to report.

The metric that should be reported for this dataset will be determined by the need that Shopify has for their business analysis. If they are looking for the most profitable store ID, the metric that would fit best would be max. median order value. If they are looking for the lowest performing store ID, that can also be determined by the store ID with the fewest sales and revenue. I feel that these metrics would give a better overall picture of how business looked in this dataset from March 2017. There are too many factors to provide simply one metric. A full picture needs to be shared to ensure quality business analysis.

However, we can look at Median Order Value(MOV) and this can provide us with better insights.


### 3) What is its value?

In [52]:
def metrics_calculator(df):
    """
    A simple function  calculates the AOV, Standard Deviation of AOV and Median Order Value

    Parameters
    ----------
    Pandas Dataframe with column 'total_items' and 'order_amount'

    Returns
    -------
    Pandas DataFrame
    """

    #calculate the AOV
    groupbyTotalItems = df.groupby('total_items')['order_amount'].mean().rename_axis('Total Items in Order').reset_index(name='AOV')

    # next we calculate the SD for AOV
    SD = df.groupby('total_items')['order_amount'].std().rename_axis('Total Items in Order').reset_index(name='std')['std']

    # calculate the Median Order Value
    median =  df.groupby('total_items')['order_amount'].median().rename_axis('Total Items in Order').reset_index(name='Median')['Median']

    #adding the SD and Median Order Value column to the dataframe
    groupbyTotalItems['Median Order Value'] = median
    groupbyTotalItems['Standard Deviation'] = SD

    return groupbyTotalItems

In [53]:
metrics_calculator(data)

,Total Items in Order,AOV,Median Order Value,Standard Deviation
0,1,417.364481,153.0,2593.090627
1,2,750.215066,306.0,4760.572162
2,3,1191.076514,459.0,7471.160149
3,4,947.686007,592.0,5977.632918
4,5,759.350649,765.0,161.174453
5,6,17940.000000,948.0,51153.864136
6,8,1064.000000,1064.0,NaN
7,2000,704000.000000,704000.0,0.000000


As we see in the data above, there are numerous orders made with anywhere from 1-2000 items in each order. To properly report the correct AOV, we need to order the data by total number of items in the order, then observe the median order value as it increases or decreases with an increase in total number of items per order.

Median Order Value (MOV) proves to be a better metric to evaluate the data.

## Conclusion

We can gather insights from the above analysis of the dataset from March 2017 to make better informed business decisions going forward. When we look at the MOV, we see that for orders with only 1 item, the median order value is $153, which makes more sense when we consider that the product is sneakers- it matches the price point we are considering.

Shopify can use these insights to isolate both poor and top performing stores with various metrics as well, to motivate and reward these individual stores. Targeted company promotions can be made with better understanding of the specific store needs. For example, a store that sells in bulk like store ID 42 will need a very different approach to a store that sells small orders of 1-3 items per order.

## Question 2: SQL- Speedy Express

Dataset: https://www.w3schools.com/SQL/TRYSQL.ASP?FILENAME=TRYSQL_SELECT_ALL

This dataset is being hosted by the w3schools website. An overview of the database reveals that there are 8 tablenames:
Customers
Categories
Employees
Order Details
Orders
Products
Shippers
Suppliers

Problem:
1) How many orders were shipped by Speedy Express in total?
2) What is the last name of the employee with the most orders?
3) What product was ordered the most by customers in Germany?


## 1) How many orders were shipped by Speedy Express in total?

**There are 54 orders shipped by Speedy Express.**

In [ ]:
SELECT ShipperName, COUNT(Orders.ShipperID)
FROM Orders
INNER JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
GROUP BY Orders.ShipperID;

## 2) What is the last name of the employee with the most orders?

**The last name of the employee with the most orders is "Peacock".**

In [ ]:
SELECT LastName, COUNT(Orders.EmployeeID)
FROM Orders
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
GROUP BY Orders.EmployeeID
Order by COUNT(Orders.EmployeeID) Desc;

## 3) What product was ordered the most by customers in Germany?

**The product that was ordered the most by customers in Germany was Gorgonzola Telino, which was ordered 5 times.**

In [ ]:
SELECT ProductName, MAX(ProductCount)
FROM (
	  SELECT Products.ProductName as ProductName, COUNT(Products.ProductName) as ProductCount
	  FROM Orders
	  INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID
	  INNER JOIN OrderDetails ON Orders.OrderID=OrderDetails.OrderID
	  INNER JOIN Products ON Products.ProductID=OrderDetails.ProductID
	  WHERE Country = 'Germany'
	  GROUP BY Products.ProductName
	  Order by COUNT(Products.ProductName) Desc
      );